In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib
import numpy
import pickle
import cartopy
import rasterio
import glob

In [2]:
from mpl_toolkits.basemap import Basemap

In [3]:
data_dir = '/tablespace/xcal/'

In [4]:
with open(f'{data_dir}/pickles/largest_20.pickle', 'rb') as f:
    largest = pickle.load(f)

with open(f'{data_dir}/pickles/timestamps.pickle', 'rb') as f:
    timestamps = pickle.load(f)

In [5]:
largest = numpy.ma.masked_where(largest==0, largest)

In [6]:
label_names, counts = numpy.unique(largest, return_counts=True)
pandas.DataFrame({'label': label_names, 'size': counts})

KeyboardInterrupt: 

# Dump PNGs

In [7]:
def get_geotiff(timestamp, folder):
    ts = timestamp.strftime('%Y-%m-%d_%H%M')    
    file_path = glob.glob(f'{folder}/GOES16-ABI-CONUS-{ts}*.tif')[0]    
    with rasterio.open(file_path) as src:
        raster = src.read()
        extent = (src.bounds[0], src.bounds[2], src.bounds[1], src.bounds[3])
    raster = numpy.ma.masked_where(raster==0, raster)
    raster = numpy.transpose(raster, (1, 2, 0))
    return raster, extent

## Conus

## Pacus

In [ ]:
proj = cartopy.crs.PlateCarree()
proj = cartopy.crs.epsg(2163) # US national Atlas equa area
opts = {'projection': proj}
fig, ax = plt.subplots(figsize=(10, 5), dpi=300, subplot_kw=opts)

plot_data = largest

folder = '/tablespace/xcal/geoColor/pacus/'

timestamp = timestamps[0]
raster, extent = get_geotiff(timestamp, folder)
goes = ax.imshow(raster, origin='upper', extent=extent, transform=proj)
ccl = ax.imshow(plot_data[0], cmap='tab20', vmin=1, vmax=21, interpolation='none', extent=[-180, 180, -90, 90], transform=proj)

ax.set_title(timestamp)  
ax.set_xlim(extent[0], extent[1])
ax.set_ylim(extent[2], extent[3])

n_labels = 20
cbar = fig.colorbar(ccl, ax=ax, orientation='vertical')
cbar.set_ticks(numpy.arange(1.5, n_labels+1.5,1) , labels=numpy.arange(1, n_labels+1, 1))

for i in range(plot_data.shape[0])[0:1]:
    timestamp = timestamps[i]
    print(timestamp)
    try:
        raster, extent = get_geotiff(timestamp)
    except:
        print(f'No geotiff for {timestamp}')
    goes.set_array(raster)
    ccl.set_array(plot_data[i])
    
    ax.set_title(timestamps[i])    
    fig.savefig(f'{data_dir}/animations/ccl_pacus/{timestamps[i]}.png', bbox_inches='tight')

## Global

In [ ]:
proj = cartopy.crs.PlateCarree()
opts = {'projection': proj}

fig, ax = plt.subplots(figsize=(10, 5), dpi=300, subplot_kw=opts)

plot_data = largest

m = Basemap(projection='cyl')
m.bluemarble(alpha=0.7)
m.drawparallels(numpy.arange(-90.,120.,30.), linewidth=0.1)
m.drawmeridians(numpy.arange(0, 360, 30), linewidth=0.1)

ccl = ax.imshow(plot_data[0], cmap='tab20', vmin=1, vmax=21, interpolation='none', extent=[-180, 180, -90, 90], transform=proj)

n_labels = 20
cbar = fig.colorbar(im, ax=ax, orientation='vertical')
cbar.set_ticks(numpy.arange(1.5, n_labels+1.5,1) , labels=numpy.arange(1, n_labels+1, 1))

for i in range(plot_data.shape[0]):
    timestamp = timestamps[i]
    print(timestamp)
    ccl.set_array(plot_data[i])    
    ax.set_title(timestamp)    
    fig.savefig(f'{data_dir}/animations/ccl/{timestamps[i]}.png', bbox_inches='tight')